# NoETL Playbook Execution Report
__Agent 007 mission report:__ _Operation completed. Martini status: shaken, not stirred._
- This notebook provides a report of the execution of a NoETL playbook using the `agent007.py` script.
- It analyzes the DuckDB database generated during the playbook run, with execution flow, step results, loop iterations, task executions, and context data captured throughout the process.
1. Run playbook with `agent007.py`.
2. Set `db_path` below to that file path.
3. Run all paragraphs to inspect tables and data.


In [1]:
%reload_ext sql


The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

## Set the database path and list all tables


In [2]:
import duckdb
import polars as pl
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
import networkx as nx
from datetime import datetime

# Path to DuckDB database
db_path = '../data/noetldb/agent007.duckdb'  # Default path used by agent007.py
# Example: db_path = '/Users/kadyapam/projects/noetl/noetl/noetl/runtime/agent007.duckdb'

if not os.path.exists(db_path) and db_path != ':memory:':
    raise FileNotFoundError(f"DuckDB database not found: {db_path}")

con = duckdb.connect(db_path)

def parse_json_column(df, column_name):
    if column_name in df.columns:
        def safe_json_parse(x):
            if not x:
                return None
            try:
                return json.loads(x)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse JSON: {e}")
                return x

        return df.with_columns([
            pl.col(column_name).map_elements(safe_json_parse, return_dtype=pl.Object)
        ])
    return df

# execution ID
latest_execution = con.execute("""
    SELECT execution_id, MAX(timestamp) as latest_time
    FROM event_log
    GROUP BY execution_id
    ORDER BY latest_time DESC
    LIMIT 1
""").fetchone()

if latest_execution:
    execution_id = latest_execution[0]
    print(f"Analyzing execution ID: {execution_id}")
else:
    print("No executions found in the database.")
    execution_id = None


Analyzing execution ID: 7544fa0b-025c-470d-82da-431c6da3f9d1


In [3]:
tables = con.execute("SHOW TABLES").fetchall()
for t in tables:
    print(t[0])


context
event_log
loop_state
step_result
task_result
transition
workbook
workflow


## Execution Overview

This section provides a high-level overview of the playbook execution.


In [4]:
if execution_id:
    # execution start end times
    execution_times = con.execute("""
        SELECT 
            MIN(timestamp) as start_time,
            MAX(timestamp) as end_time,
            (EXTRACT(EPOCH FROM MAX(timestamp)) - EXTRACT(EPOCH FROM MIN(timestamp))) as duration_seconds
        FROM event_log
        WHERE execution_id = ?
    """, [execution_id]).fetchone()

    # execution statistics
    execution_stats = con.execute("""
        SELECT
            (SELECT COUNT(*) FROM step_result WHERE execution_id = ?) as step_count,
            (SELECT COUNT(*) FROM task_result WHERE execution_id = ?) as task_count,
            (SELECT COUNT(*) FROM loop_state WHERE execution_id = ?) as loop_count,
            (SELECT COUNT(*) FROM event_log WHERE execution_id = ? AND event_type LIKE '%error%') as error_count
    """, [execution_id, execution_id, execution_id, execution_id]).fetchone()

    # playbook name
    playbook_info = con.execute("""
        SELECT node_name, metadata
        FROM event_log
        WHERE execution_id = ? AND event_type = 'execution_start'
        LIMIT 1
    """, [execution_id]).fetchone()

    # execution overview
    print(f"Playbook: {playbook_info[0] if playbook_info else 'Unknown'}")
    print(f"Start Time: {execution_times[0]}")
    print(f"End Time: {execution_times[1]}")
    print(f"Duration: {execution_times[2]:.2f} seconds")
    print(f"Steps Executed: {execution_stats[0]}")
    print(f"Tasks Executed: {execution_stats[1]}")
    print(f"Loops Executed: {execution_stats[2]}")
    print(f"Errors Encountered: {execution_stats[3]}")

    # playbook metadata
    if playbook_info and playbook_info[1]:
        try:
            metadata = json.loads(playbook_info[1])
            print("\nPlaybook Metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        except:
            pass
else:
    print("No execution ID available.")


Playbook: weather_iterator_example
Start Time: 2025-06-15 23:50:05.167009
End Time: 2025-06-15 23:50:07.516633
Duration: 2.35 seconds
Steps Executed: 4
Tasks Executed: 6
Loops Executed: 1
Errors Encountered: 0

Playbook Metadata:
  playbook_path: ./catalog/playbooks/weather_example.yaml


## Step Execution Analysis

Analyze the execution of steps in the playbook.


In [5]:
if execution_id:
    # step execution details
    step_results = pl.from_arrow(con.execute("""
        SELECT 
            step_id, 
            step_name, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM step_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    step_results = parse_json_column(step_results, 'data')

    # step execution summary
    step_summary = pl.from_arrow(con.execute("""
        SELECT 
            step_name, 
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM step_result
        WHERE execution_id = ?
        GROUP BY step_name
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Step Execution Summary:")
    display(step_summary)

    # step results
    print("\nStep Results:")
    display(step_results)
else:
    print("No execution ID available.")


Step Execution Summary:


step_name,execution_count,success_count,error_count
str,i64,i64,i64
"""city_loop""",1,1,0
"""end_city_loop""",1,1,0
"""start""",1,1,0
"""aggregate_alerts""",1,1,0



Step Results:


shape: (4, 7)
┌────────────────┬───────────────┬───────────────┬───────────────┬─────────┬───────────────┬───────┐
│ step_id        ┆ step_name     ┆ parent_id     ┆ timestamp     ┆ status  ┆ data          ┆ error │
│ ---            ┆ ---           ┆ ---           ┆ ---           ┆ ---     ┆ ---           ┆ ---   │
│ str            ┆ str           ┆ str           ┆ datetime[μs]  ┆ str     ┆ object        ┆ str   │
╞════════════════╪═══════════════╪═══════════════╪═══════════════╪═════════╪═══════════════╪═══════╡
│ 931259dc-787a- ┆ start         ┆ null          ┆ 2025-06-15    ┆ success ┆ {}            ┆ null  │
│ 46e0-91f0-0636 ┆               ┆               ┆ 23:50:05.1820 ┆         ┆               ┆       │
│ c8…            ┆               ┆               ┆ 60            ┆         ┆               ┆       │
│ 8c9ea2e1-2f4c- ┆ city_loop     ┆ null          ┆ 2025-06-15    ┆ success ┆ [{'fetch_and_ ┆ null  │
│ 492b-ac9a-0471 ┆               ┆               ┆ 23:50:07.4866 ┆         ┆ evaluate':    ┆       │
│ 26…            ┆               ┆               ┆ 71            ┆         ┆ {'data…       ┆       │
│ cf21b5a1-c881- ┆ end_city_loop ┆ 6127accb-cd8e ┆ 2025-06-15    ┆ success ┆ {'alerts':    ┆ null  │
│ 45b6-9638-ad42 ┆               ┆ -4cd3-80e5-55 ┆ 23:50:07.5029 ┆         ┆ [{'fetch_and_ ┆       │
│ 23…            ┆               ┆ 498b…         ┆ 12            ┆         ┆ evalua…       ┆       │
│ 8384607b-5069- ┆ aggregate_ale ┆ null          ┆ 2025-06-15    ┆ success ┆ {}            ┆ null  │
│ 495e-9dad-1d4b ┆ rts           ┆               ┆ 23:50:07.5119 ┆         ┆               ┆       │
│ 70…            ┆               ┆               ┆ 32            ┆         ┆               ┆       │
└────────────────┴───────────────┴───────────────┴───────────────┴─────────┴───────────────┴───────┘

## Loop Execution Analysis

Analyze the execution of loops in the playbook.


In [6]:
if execution_id:
    # loop execution details
    loop_state = pl.from_arrow(con.execute("""
        SELECT 
            loop_id, 
            loop_name, 
            parent_id, 
            iterator, 
            items, 
            current_index, 
            current_item, 
            results, 
            timestamp, 
            status
        FROM loop_state
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    loop_state = parse_json_column(loop_state, 'items')
    loop_state = parse_json_column(loop_state, 'current_item')
    loop_state = parse_json_column(loop_state, 'results')

    # loop execution summary
    if len(loop_state) > 0:
        print("Loop Execution Summary:")
        display(loop_state)

        # loop iteration events
        loop_events = pl.from_arrow(con.execute("""
            SELECT 
                event_id, 
                node_name, 
                event_type, 
                timestamp, 
                status, 
                duration, 
                output_result as data, 
                metadata
            FROM event_log
            WHERE execution_id = ? AND event_type LIKE '%loop%'
            ORDER BY timestamp
        """, [execution_id]).arrow())

        loop_events = parse_json_column(loop_events, 'data')
        loop_events = parse_json_column(loop_events, 'metadata')

        print("\nLoop Events:")
        display(loop_events)
    else:
        print("No loop executions found.")
else:
    print("No execution ID available.")


Loop Execution Summary:


shape: (1, 10)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ loop_id   ┆ loop_name ┆ parent_id ┆ iterator ┆ … ┆ current_i ┆ results   ┆ timestamp ┆ status    │
│ ---       ┆ ---       ┆ ---       ┆ ---      ┆   ┆ tem       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ str       ┆ str       ┆ str      ┆   ┆ ---       ┆ object    ┆ datetime[ ┆ str       │
│           ┆           ┆           ┆          ┆   ┆ object    ┆           ┆ μs]       ┆           │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 6127accb- ┆ city_loop ┆ 8c9ea2e1- ┆ city     ┆ … ┆ null      ┆ [{'fetch_ ┆ 2025-06-1 ┆ completed │
│ cd8e-4cd3 ┆           ┆ 2f4c-492b ┆          ┆   ┆           ┆ and_evalu ┆ 5 23:50:0 ┆           │
│ -80e5-554 ┆           ┆ -ac9a-047 ┆          ┆   ┆           ┆ ate':     ┆ 7.502612  ┆           │
│ 98b…      ┆           ┆ 126…      ┆          ┆   ┆           ┆ {'data…   ┆           ┆           │
└───────────┴───────────┴───────────┴──────────┴───┴───────────┴───────────┴───────────┴───────────┘


Loop Events:


shape: (10, 8)
┌────────────┬────────────┬────────────┬────────────┬───────────┬──────────┬───────────┬───────────┐
│ event_id   ┆ node_name  ┆ event_type ┆ timestamp  ┆ status    ┆ duration ┆ data      ┆ metadata  │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---       ┆ ---      ┆ ---       ┆ ---       │
│ str        ┆ str        ┆ str        ┆ datetime[μ ┆ str       ┆ f64      ┆ object    ┆ object    │
│            ┆            ┆            ┆ s]         ┆           ┆          ┆           ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══════════╪══════════╪═══════════╪═══════════╡
│ f5551caa-9 ┆ city_loop  ┆ loop_start ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'item_co │
│ 419-4e61-b ┆            ┆            ┆ 23:50:05.1 ┆ ss        ┆          ┆           ┆ unt': 3,  │
│ edd-8e731c ┆            ┆            ┆ 93496      ┆           ┆          ┆           ┆ 'iterator │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ ': …      │
│ 9c96be44-c ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ 142-4a16-9 ┆ 0]         ┆ tion       ┆ 23:50:05.1 ┆ ss        ┆          ┆           ┆ 0,        │
│ e55-3c4afa ┆            ┆            ┆ 98216      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 0ef2b7e4-8 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ a77-43dd-8 ┆ 0]         ┆ tion_compl ┆ 23:50:06.0 ┆           ┆          ┆ nd_evalua ┆ 0,        │
│ 326-5f0a81 ┆            ┆ ete        ┆ 34029      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 737e5dba-5 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ 99d-4029-b ┆ 1]         ┆ tion       ┆ 23:50:06.0 ┆ ss        ┆          ┆           ┆ 1,        │
│ a25-7c8ccf ┆            ┆            ┆ 35916      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ a2791d86-6 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ 0db-45fe-a ┆ 1]         ┆ tion_compl ┆ 23:50:06.7 ┆           ┆          ┆ nd_evalua ┆ 1,        │
│ c6c-32045f ┆            ┆ ete        ┆ 48961      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ a9d41d1e-7 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ ce4-40a2-b ┆ 2]         ┆ tion       ┆ 23:50:06.7 ┆ ss        ┆          ┆           ┆ 2,        │
│ 8f3-5e72cb ┆            ┆            ┆ 50926      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 37380f4f-b ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ eb4-402a-b ┆ 2]         ┆ tion_compl ┆ 23:50:07.4 ┆           ┆          ┆ nd_evalua ┆ 2,        │
│ b0d-855c8b ┆            ┆ ete        ┆ 84151      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆      

## Task Execution Analysis

Analyze the execution of tasks in the playbook.


In [7]:
if execution_id:
    # task execution details
    task_results = pl.from_arrow(con.execute("""
        SELECT 
            task_id, 
            task_name, 
            task_type, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM task_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    task_results = parse_json_column(task_results, 'data')

    # task execution summary
    task_summary = pl.from_arrow(con.execute("""
        SELECT 
            task_name, 
            task_type,
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM task_result
        WHERE execution_id = ?
        GROUP BY task_name, task_type
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Task Execution Summary:")
    display(task_summary)

    # task results
    print("\nTask Results:")
    display(task_results)
else:
    print("No execution ID available.")


Task Execution Summary:


task_name,task_type,execution_count,success_count,error_count
str,str,i64,i64,i64
"""get_forecast""","""http""",3,3,0
"""evaluate_weather""","""python""",3,3,0



Task Results:


task_id,task_name,task_type,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,object,str
"""7959e909-dc36-406a-8d7b-4203dc…","""get_forecast""","""http""","""a6a7c40d-067d-442a-955a-6e8ed5…",2025-06-15 23:50:06.007793,"""success""","{'data': {'latitude': 51.5, 'longitude': -0.120000124, 'generationtime_ms': 0.02467632293701172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [17.1, 16.5, 16.1, 15.5, 15.1, 14.9, 15.4, 16.5, 17.9, 19.4, 20.6, 21.8, 22.8, 23.6, 24.8, 25.1, 25.5, 25.8, 25.5, 23.7, 22.1, 21.2, 20.2, 19.4]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 06:50:05 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",null
"""266c04d2-7574-4bef-9b77-e0b480…","""evaluate_weather""","""python""","""6baf936b-57b4-4b0b-9463-a920d6…",2025-06-15 23:50:06.026430,"""success""","{'city': 'London', 'max_temp': 25.8, 'alert': True}",null
"""bd191b5f-47eb-47e3-9aa6-29e16a…","""get_forecast""","""http""","""e16aa6fa-63c6-4252-a1b4-5a27df…",2025-06-15 23:50:06.715634,"""success""","{'data': {'latitude': 48.84, 'longitude': 2.3599997, 'generationtime_ms': 0.026702880859375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 46.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [16.3, 15.7, 15.2, 14.5, 14.1, 14.1, 14.6, 15.6, 17.0, 18.7, 20.5, 22.0, 23.3, 24.1, 25.1, 25.4, 26.0, 25.5, 25.2, 24.4, 23.7, 22.8, 21.9, 21.3]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 06:50:06 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",null
"""52866d85-9043-48d7-846c-6c6454…","""evaluate_weather""","""python""","""8fdc1437-4098-4873-af5d-54e027…",2025-06-15 23:50:06.740597,"""success""","{'city': 'Paris', 'max_temp': 26.0, 'alert': True}",null
"""1d51465f-bcf7-443e-9ae7-934990…","""get_forecast""","""http""","""c7a98778-6096-440e-9fdc-4cac3b…",2025-06-15 23:50:07.450734,"""success""","{'data': {'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.02467632293701172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperatur

## Context Analysis

Analyze the context data during playbook execution.


In [8]:
if execution_id:
    # context data
    context_data = pl.from_arrow(con.execute("""
        SELECT 
            key, 
            value, 
            timestamp
        FROM context
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    context_data = parse_json_column(context_data, 'value')

    if len(context_data) > 0:
        print("Context Data:")
        display(context_data)

        # context state
        final_context = {}
        for _, row in context_data.to_pandas().iterrows():
            final_context[row['key']] = row.get('value')

        print("\nFinal Context State:")
        for key, value in final_context.items():
            print(f"\n{key}:")
            if isinstance(value, (dict, list)):
                print(json.dumps(value, indent=2))
            else:
                print(value)
    else:
        print("No context data found.")
else:
    print("No execution ID found.")


Context Data:


key,value,timestamp
str,object,datetime[μs]
"""workload""","{'jobId': '{{ job.uuid }}', 'state': 'ready', 'cities': [{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}], 'base_url': 'https://api.open-meteo.com/v1', 'temperature_threshold': 25}",2025-06-15 23:50:05.164172
"""execution_start""",2025-06-15T23:50:05.166234,2025-06-15 23:50:05.166249
"""base_url""",https://api.open-meteo.com/v1,2025-06-15 23:50:06.753418
"""get_forecast""","{'data': {'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.02467632293701172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 06:50:07 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",2025-06-15 23:50:07.460374
"""get_forecast.result""","{'data': {'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.02467632293701172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 06:50:07 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",2025-06-15 23:50:07.461225
…,…,…
"""evaluate_weather.result""","{'city': 'Berlin', 'max_temp': 22.7, 'alert': False}",2025-06-15 23:50:07.480753
"""evaluate_weather.status""",success,2025-06-15 23:50:07.481069
"""result""","{'city': 'Berlin', 'max_temp': 22.7, 'alert': False}",2025-06-15 23:50:07.481440



Final Context State:

workload:
{
  "jobId": "{{ job.uuid }}",
  "state": "ready",
  "cities": [
    {
      "name": "London",
      "lat": 51.51,
      "lon": -0.13
    },
    {
      "name": "Paris",
      "lat": 48.85,
      "lon": 2.35
    },
    {
      "name": "Berlin",
      "lat": 52.52,
      "lon": 13.41
    }
  ],
  "base_url": "https://api.open-meteo.com/v1",
  "temperature_threshold": 25
}

execution_start:
2025-06-15T23:50:05.166234

base_url:
https://api.open-meteo.com/v1

get_forecast:
{
  "data": {
    "latitude": 52.52,
    "longitude": 13.419998,
    "generationtime_ms": 0.02467632293701172,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 38.0,
    "hourly_units": {
      "time": "iso8601",
      "temperature_2m": "\u00b0C"
    },
    "hourly": {
      "time": [
        "2025-06-16T00:00",
        "2025-06-16T01:00",
        "2025-06-16T02:00",
        "2025-06-16T03:00",
        "2025-06-16T04:00",
        "2

## Preview `event_log` table


In [9]:
table_name = 'event_log'
event_log_df = pl.from_arrow(con.execute(f"SELECT * FROM {table_name} LIMIT 100").arrow())
event_log_df


execution_id,event_id,parent_event_id,timestamp,event_type,node_id,node_name,node_type,status,duration,input_context,output_result,metadata
str,str,str,datetime[μs],str,str,str,str,str,f64,str,str,str
"""a497e03e-1e8a-4708-97da-cd981e…","""11d58c32-68b0-45e6-818a-295d7a…",null,2025-06-15 21:06:41.543364,"""execution_start""","""a497e03e-1e8a-4708-97da-cd981e…","""weather_iterator_example""","""playbook""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""playbook_path"": ""catalog/pla…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0992c224-020a-40d0-ad9d-836927…",null,2025-06-15 21:06:41.544493,"""step_start""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""bf7026af-a965-496e-99cf-eda7f0…","""0992c224-020a-40d0-ad9d-836927…",2025-06-15 21:06:41.545576,"""step_complete""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""success""",0.001186,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{}""","""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""6193ea67-8155-4e2b-b6be-4de1b3…","""11d58c32-68b0-45e6-818a-295d7a…",2025-06-15 21:06:41.549975,"""step_transition""","""a497e03e-1e8a-4708-97da-cd981e…","""transition_to_city_loop""","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""start"", ""to_ste…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0991095f-57c4-45a8-b17f-c0c999…",null,2025-06-15 21:06:41.551111,"""step_start""","""3f85e793-7aa8-4823-b0c2-7c7533…","""city_loop""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""a497e03e-1e8a-4708-97da-cd981e…","""e109a041-5a5e-457c-93bf-df2922…","""5a14bb1b-a100-4946-be87-d67dc8…",2025-06-15 21:06:41.649862,"""task_complete""","""0d434aa7-5fa8-464d-9ce6-3b4f1e…","""process_district""","""task.python""","""success""",0.002933,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{""city"": ""Berlin"", ""district"":…","""{""task_type"": ""python""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""1901af17-4fb0-4e3d-805f-1dcefb…","""25965b28-8a60-4245-8a92-a54216…",2025-06-15 21:06:41.652961,"""step_complete""","""6db4aea4-59fd-4c1d-adb6-055eb2…","""process_district""","""step""","""success""",0.007206,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{""city"": ""Berlin"", ""district"":…","""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""c66b2ee1-4241-4476-b0e1-757300…","""11d58c32-68b0-45e6-818a-295d7a…",2025-06-15 21:06:41.653542,"""step_transition""","""a497e03e-1e8a-4708-97da-cd981e…","""transition_to_end_district_loo…","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""process_distric…"


## Steps resluts


In [10]:
pl.from_arrow(con.execute("SELECT * FROM step_result ORDER BY timestamp DESC LIMIT 10").arrow())


execution_id,step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,str,str
"""7544fa0b-025c-470d-82da-431c6d…","""8384607b-5069-495e-9dad-1d4b70…","""aggregate_alerts""",null,2025-06-15 23:50:07.511932,"""success""","""{}""",null
"""7544fa0b-025c-470d-82da-431c6d…","""cf21b5a1-c881-45b6-9638-ad4223…","""end_city_loop""","""6127accb-cd8e-4cd3-80e5-55498b…",2025-06-15 23:50:07.502912,"""success""","""{""alerts"": [{""fetch_and_evalua…",null
"""7544fa0b-025c-470d-82da-431c6d…","""8c9ea2e1-2f4c-492b-ac9a-047126…","""city_loop""",null,2025-06-15 23:50:07.486671,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""7544fa0b-025c-470d-82da-431c6d…","""931259dc-787a-46e0-91f0-0636c8…","""start""",null,2025-06-15 23:50:05.182060,"""success""","""{}""",null
"""d6ec3664-77cb-4bfe-a907-6b52b8…","""49dbdbec-b161-44a9-93ef-3954b6…","""aggregate_alerts""",null,2025-06-15 23:40:07.548055,"""success""","""{}""",null
"""d6ec3664-77cb-4bfe-a907-6b52b8…","""5ad35673-60e0-4e4c-a160-fd0ff5…","""end_city_loop""","""6da9f570-22de-4651-a621-f1d4cf…",2025-06-15 23:40:07.540755,"""success""","""{""alerts"": [{""fetch_and_evalua…",null
"""d6ec3664-77cb-4bfe-a907-6b52b8…","""e59ecff0-1f95-4754-b68b-746571…","""city_loop""",null,2025-06-15 23:40:07.525196,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""d6ec3664-77cb-4bfe-a907-6b52b8…","""f1ed0ec6-3bb4-4090-a315-48e372…","""start""",null,2025-06-15 23:40:05.241986,"""success""","""{}""",null
"""57ec3809-bf1d-4e28-afc8-085549…","""b605fb27-0dcd-4d01-b0c2-1863d4…","""aggregate_alerts""",null,2025-06-15 23:38:17.528588,"""success""","""{}""",null


## Task results

In [11]:
con.sql("SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────┬───────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
%sql con --alias duckdb
%sql SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10

Running query in 'duckdb'

execution_id,task_id,task_name,task_type,parent_id,timestamp,status,data,error
7544fa0b-025c-470d-82da-431c6da3f9d1,a0441290-dab4-4dd8-bf48-b7c2be8b0970,evaluate_weather,python,0c74bac5-6543-4640-bde7-188e0d69722e,2025-06-15 23:50:07.475482,success,"{""city"": ""Berlin"", ""max_temp"": 22.7, ""alert"": false}",None
7544fa0b-025c-470d-82da-431c6da3f9d1,1d51465f-bcf7-443e-9ae7-934990f71b58,get_forecast,http,c7a98778-6096-440e-9fdc-4cac3bb23d3a,2025-06-15 23:50:07.450734,success,"{""data"": {""latitude"": 52.52, ""longitude"": 13.419998, ""generationtime_ms"": 0.02467632293701172, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 38.0, ""hourly_units"": {""time"": ""iso8601"", ""temperature_2m"": ""\u00b0C""}, ""hourly"": {""time"": [""2025-06-16T00:00"", ""2025-06-16T01:00"", ""2025-06-16T02:00"", ""2025-06-16T03:00"", ""2025-06-16T04:00"", ""2025-06-16T05:00"", ""2025-06-16T06:00"", ""2025-06-16T07:00"", ""2025-06-16T08:00"", ""2025-06-16T09:00"", ""2025-06-16T10:00"", ""2025-06-16T11:00"", ""2025-06-16T12:00"", ""2025-06-16T13:00"", ""2025-06-16T14:00"", ""2025-06-16T15:00"", ""2025-06-16T16:00"", ""2025-06-16T17:00"", ""2025-06-16T18:00"", ""2025-06-16T19:00"", ""2025-06-16T20:00"", ""2025-06-16T21:00"", ""2025-06-16T22:00"", ""2025-06-16T23:00""], ""temperature_2m"": [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, ""status_code"": 200, ""headers"": {""date"": ""Mon, 16 Jun 2025 06:50:07 GMT"", ""content-type"": ""application/json; charset=utf-8"", ""transfer-encoding"": ""chunked"", ""connection"": ""keep-alive"", ""content-encoding"": ""deflate""}}",None
7544fa0b-025c-470d-82da-431c6da3f9d1,52866d85-9043-48d7-846c-6c64544e7e32,evaluate_weather,python,8fdc1437-4098-4873-af5d-54e027b74513,2025-06-15 23:50:06.740597,success,"{""city"": ""Paris"", ""max_temp"": 26.0, ""alert"": true}",None
7544fa0b-025c-470d-82da-431c6da3f9d1,bd191b5f-47eb-47e3-9aa6-29e16a090623,get_forecast,http,e16aa6fa-63c6-4252-a1b4-5a27dfbf02f3,2025-06-15 23:50:06.715634,success,"{""data"": {""latitude"": 48.84, ""longitude"": 2.3599997, ""generationtime_ms"": 0.026702880859375, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 46.0, ""hourly_units"": {""time"": ""iso8601"", ""temperature_2m"": ""\u00b0C""}, ""hourly"": {""time"": [""2025-06-16T00:00"", ""2025-06-16T01:00"", ""2025-06-16T02:00"", ""2025-06-16T03:00"", ""2025-06-16T04:00"", ""2025-06-16T05:00"", ""2025-06-16T06:00"", ""2025-06-16T07:00"", ""2025-06-16T08:00"", ""2025-06-16T09:00"", ""2025-06-16T10:00"", ""2025-06-16T11:00"", ""2025-06-16T12:00"", ""2025-06-16T13:00"", ""2025-06-16T14:00"", ""2025-06-16T15:00"", ""2025-06-16T16:00"", ""2025-06-16T17:00"", ""2025-06-16T18:00"", ""2025-06-16T19:00"", ""2025-06-16T20:00"", ""2025-06-16T21:00"", ""2025-06-16T22:00"", ""2025-06-16T23:00""], ""temperature_2m"": [16.3, 15.7, 15.2, 14.5, 14.1, 14.1, 14.6, 15.6, 17.0, 18.7, 20.5, 22.0, 23.3, 24.1, 25.1, 25.4, 26.0, 25.5, 25.2, 24.4, 23.7, 22.8, 21.9, 21.3]}}, ""status_code"": 200, ""headers"": {""date"": ""Mon, 16 Jun 2025 06:50:06 GMT"", ""content-type"": ""application/json; charset=utf-8"", ""transfer-encoding"": ""chunked"", ""connection"": ""keep-alive"", ""content-encoding"": ""deflate""}}",None
7544fa0b-025c-470d-82da-431c6da3f9d1,266c04d2-7574-4bef-9b77-e0b480534f61,evaluate_weather,python,6baf936b-57b4-4b0b-9463-a920d6bb8679,2025-06-15 23:50:06.026430,success,"{""city"": ""London"", ""max_temp"": 25.8, ""alert"": true}",None
7544fa0b-025c-470d-82da-431c6da3f9d1,7959e909-dc36-406a-8d7b-4203dcc91c21,get_forecast,http,a6a7c40d-067d-442a-955a-6e8ed595719e,2025-06-15 23:50:06.007793,success,"{""data"": {""latitude"": 51.5, ""longitude"": -0.120000124, ""generationtime_ms"": 0.02467632293701172, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 29.0,